In [3]:
# P25 — Informe final (master table + MD blocks + opcional figuras)

from pathlib import Path
import json, numpy as np, pandas as pd

# Monta Drive si hace falta
try:
    from google.colab import drive
    drive.mount('/content/drive')
except Exception:
    pass

BASE = Path("/content/drive/MyDrive/CognitivaAI")
OUT  = BASE/"p25_informe_final"
OUT.mkdir(parents=True, exist_ok=True)

def safe_exists(p):
    try: return Path(p).exists()
    except: return False

print("BASE:", BASE)
print("OUT :", OUT)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
BASE: /content/drive/MyDrive/CognitivaAI
OUT : /content/drive/MyDrive/CognitivaAI/p25_informe_final


In [4]:
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss

def _clean_cols(df):
    def cleanse(s): return str(s).replace('\ufeff','').strip()
    df.columns = [cleanse(c) for c in df.columns]
    return df

def infer_cohort_from_pid(pid):
    s = str(pid).strip().upper()
    if s.startswith("OAS1"): return "OAS1"
    if s.startswith("OAS2"): return "OAS2"
    return "ALL"

def load_csv(path):
    try:
        df = pd.read_csv(path, encoding="utf-8-sig")
    except Exception:
        df = pd.read_csv(path)
    return _clean_cols(df)

def metrics_from_scores(y, p):
    y = np.asarray(y).astype(int)
    p = np.asarray(p).astype(float)
    has_var = len(np.unique(y))>1
    return dict(
        AUC   = float(roc_auc_score(y, p)) if has_var else float('nan'),
        PRAUC = float(average_precision_score(y, p)) if has_var else float('nan'),
        Brier = float(brier_score_loss(y, p))
    )

def add_cohort_col(df):
    if 'cohort' not in df.columns and 'patient_id' in df.columns:
        df = df.copy()
        df['cohort'] = df['patient_id'].map(infer_cohort_from_pid)
    return df

def detect_label_col(df):
    for c in ['y_true','y','target','label','truth','gt']:
        if c in df.columns: return c
    # última oportunidad: binaria numérica
    for c in df.columns:
        s = pd.to_numeric(df[c], errors='coerce')
        u = s.dropna().unique()
        if len(u)==2 and set(u).issubset({0,1}):
            return c
    return None

def detect_prob_col(df):
    for c in ['y_prob','y_score','prob','score','pred','prediction',
              'meta_pred','y_meta','HGB_platt','LR_platt','HGB_iso','LR_iso',
              'y_blend','y_blend_platt','y_blend_iso']:
        if c in df.columns: return c
    # primera numérica no binaria
    for c in df.columns:
        x = pd.to_numeric(df[c], errors='coerce')
        if x.notna().any() and not set(x.dropna().unique()).issubset({0,1}):
            return c
    return None


In [5]:
rows_p19 = []
included_p19 = "missing"

p19_dir = BASE/"p19_meta_ensemble"
p19_sum = p19_dir/"p19_summary.json"

def overview_rows(df, tag):
    df = add_cohort_col(df)
    ycol = detect_label_col(df); pcol = detect_prob_col(df)
    assert ycol and pcol, f"[P19 {tag}] No detecto y/prob en {list(df.columns)}"
    y = pd.to_numeric(df[ycol], errors='coerce').astype(int).values
    p = pd.to_numeric(df[pcol], errors='coerce').values
    # Global
    m_all = metrics_from_scores(y, p)
    out = [dict(Pipeline="P19", Split=tag, Cohort="ALL", **m_all, Acc=None, Precision=None, Recall=None, Thr=None, Cost=None, Notas="meta-XGB")]
    # Cohortes
    for coh in ["OAS1","OAS2"]:
        idx = (df['cohort'].values==coh)
        if idx.sum()>1:
            out.append(dict(Pipeline="P19", Split=tag, Cohort=coh, **metrics_from_scores(y[idx], p[idx]),
                            Acc=None, Precision=None, Recall=None, Thr=None, Cost=None, Notas="meta-XGB"))
    return out

if safe_exists(p19_sum):
    s = json.load(open(p19_sum))
    for split_key, split_name in [("val_overview","VAL"), ("test_overview","TEST")]:
        for e in s.get(split_key, []):
            rows_p19.append(dict(Pipeline="P19", Split=split_name, Cohort=e.get("Cohort","ALL"),
                                 AUC=e.get("AUC"), PRAUC=e.get("PRAUC"), Brier=e.get("Brier"),
                                 Acc=None, Precision=None, Recall=None, Thr=None, Cost=None, Notas=s.get("meta_model","meta")))
    included_p19 = "summary"
else:
    val_csv  = p19_dir/"p19_val_patient_preds.csv"
    test_csv = p19_dir/"p19_test_patient_preds.csv"
    any_added = False
    if safe_exists(val_csv):
        rows_p19 += overview_rows(load_csv(val_csv), "VAL"); any_added = True
    if safe_exists(test_csv):
        rows_p19 += overview_rows(load_csv(test_csv), "TEST"); any_added = True
    included_p19 = "reconstructed" if any_added else "missing"

print("P19:", included_p19, "| filas:", len(rows_p19))


P19: summary | filas: 6


In [7]:
# 🧪 Celda D — Cargador P22 (robusto: usa summary si es dict; si no, reconstruye desde CSV)
rows_p22 = []
included_p22 = "missing"

p22_dir = BASE/"p22_meta_ablation"
p22_sum = p22_dir/"p22_final_summary.json"

def reconstruct_p22_from_csv():
    local_rows = []
    val_csv  = p22_dir/"p22_val_calibrations.csv"
    test_csv = p22_dir/"p22_test_calibrations.csv"
    if safe_exists(val_csv) and safe_exists(test_csv):
        for tag, path in [("VAL",val_csv),("TEST",test_csv)]:
            df = load_csv(path)
            df = add_cohort_col(df)
            ycol = detect_label_col(df); assert ycol, f"[P22 {tag}] No y_true en {list(df.columns)}"
            cand = [c for c in ["LR_platt","LR_iso","HGB_platt","HGB_iso"] if c in df.columns]
            for c in cand:
                # Global
                local_rows.append(dict(Pipeline="P22", Split=tag, Cohort="ALL",
                                       **metrics_from_scores(df[ycol], df[c]),
                                       Acc=None, Precision=None, Recall=None, Thr=None, Cost=None, Notas=c))
                # Por cohorte
                for coh in ["OAS1","OAS2"]:
                    part = df[df['cohort']==coh]
                    if len(part)>1:
                        local_rows.append(dict(Pipeline="P22", Split=tag, Cohort=coh,
                                               **metrics_from_scores(part[ycol], part[c]),
                                               Acc=None, Precision=None, Recall=None, Thr=None, Cost=None, Notas=c))
    return local_rows

if safe_exists(p22_sum):
    try:
        s = json.load(open(p22_sum))
    except Exception:
        s = None

    used_summary = False
    if isinstance(s, dict):
        # Estructura esperada tipo {"val_best":[...], "test_best":[...]} u otros keys
        for split in ["VAL","TEST"]:
            key = f"{split.lower()}_best"
            if key in s and isinstance(s[key], list):
                for e in s[key]:
                    rows_p22.append(dict(Pipeline="P22", Split=split, Cohort=e.get("Cohort","ALL"),
                                         AUC=e.get("AUC"), PRAUC=e.get("PRAUC"), Brier=e.get("Brier"),
                                         Acc=e.get("Acc"), Precision=e.get("Precision"), Recall=e.get("Recall"),
                                         Thr=None, Cost=None, Notas=f"{e.get('Model','')}-{e.get('Calib','')}"))
                used_summary = True
        # Si no encontramos esas claves, intentamos otras convenciones comunes
        if not used_summary and isinstance(s, dict):
            for split in ["VAL","TEST"]:
                key = f"{split.lower()}_overview"
                if key in s and isinstance(s[key], list):
                    for e in s[key]:
                        rows_p22.append(dict(Pipeline="P22", Split=split, Cohort=e.get("Cohort","ALL"),
                                             AUC=e.get("AUC"), PRAUC=e.get("PRAUC"), Brier=e.get("Brier"),
                                             Acc=e.get("Acc"), Precision=e.get("Precision"), Recall=e.get("Recall"),
                                             Thr=None, Cost=None, Notas=e.get("Method") or e.get("Calib")))
                    used_summary = True
    else:
        # s es lista u otro tipo → no fiable: forzamos reconstrucción
        used_summary = False

    if used_summary and len(rows_p22)>0:
        included_p22 = "summary"
    else:
        rows_p22 = reconstruct_p22_from_csv()
        included_p22 = "reconstructed" if len(rows_p22)>0 else "missing"
else:
    rows_p22 = reconstruct_p22_from_csv()
    included_p22 = "reconstructed" if len(rows_p22)>0 else "missing"

print("P22:", included_p22, "| filas:", len(rows_p22))



P22: reconstructed | filas: 24


In [8]:
rows_p23 = []
included_p23 = "missing"

p23_dir = BASE/"p23_meta_costcohort"
p23_sum = p23_dir/"p23_summary.json"
p23_thr = p23_dir/"p23_thresholds.json"

if safe_exists(p23_sum):
    s = json.load(open(p23_sum))
    # TEST overview por cohorte/método (métricas agregadas)
    for e in s.get("test_overview", []):
        rows_p23.append(dict(Pipeline="P23", Split="TEST", Cohort=e["Cohort"],
                             AUC=e["AUC"], PRAUC=e["PRAUC"], Brier=e["Brier"],
                             Acc=e.get("Acc"), Precision=e.get("Precision"), Recall=e.get("Recall"),
                             Thr=None, Cost=e.get("Cost"), Notas=e.get("Method")))
    # Thresholds de VAL (coste-óptimo)
    if safe_exists(p23_thr):
        th = json.load(open(p23_thr))
        for k,v in th.items():
            coh, meth = k.split("_",1)
            rows_p23.append(dict(Pipeline="P23", Split="VAL", Cohort=coh,
                                 AUC=None, PRAUC=None, Brier=None, Acc=None, Precision=None, Recall=None,
                                 Thr=v.get("thr"), Cost=v.get("cost_val"), Notas=f"{meth}-cost5:1"))
    included_p23 = "summary"

print("P23:", included_p23, "| filas:", len(rows_p23))


P23: summary | filas: 8


In [9]:
rows_p24 = []
included_p24 = "missing"

p24_dir = BASE/"p24_meta_simple"
p24_sum = p24_dir/"p24_summary.json"
p24_rep = p24_dir/"p24_test_report.csv"

if safe_exists(p24_sum):
    s = json.load(open(p24_sum))
    # Global
    vg = s.get("val_global", {}); tg = s.get("test_global", {})
    if vg:
        rows_p24.append(dict(Pipeline="P24", Split="VAL", Cohort="ALL",
                             AUC=vg.get("AUC"), PRAUC=vg.get("PRAUC"), Brier=vg.get("Brier"),
                             Acc=None, Precision=None, Recall=None, Thr=None, Cost=None, Notas="Platt"))
    if tg:
        rows_p24.append(dict(Pipeline="P24", Split="TEST", Cohort="ALL",
                             AUC=tg.get("AUC"), PRAUC=tg.get("PRAUC"), Brier=tg.get("Brier"),
                             Acc=None, Precision=None, Recall=None, Thr=None, Cost=None, Notas="Platt"))
    # Por cohorte
    for e in s.get("val_by_cohort", []):
        rows_p24.append(dict(Pipeline="P24", Split="VAL", Cohort=e["Cohort"],
                             AUC=e["AUC"], PRAUC=e["PRAUC"], Brier=e["Brier"],
                             Acc=None, Precision=None, Recall=None, Thr=None, Cost=None, Notas="Platt"))
    for e in s.get("test_by_cohort", []):
        rows_p24.append(dict(Pipeline="P24", Split="TEST", Cohort=e["Cohort"],
                             AUC=e["AUC"], PRAUC=e["PRAUC"], Brier=e["Brier"],
                             Acc=None, Precision=None, Recall=None, Thr=None, Cost=None, Notas="Platt"))
    # Reporte de decisión/coste por cohorte
    if safe_exists(p24_rep):
        rep = load_csv(p24_rep)
        for _,r in rep.iterrows():
            rows_p24.append(dict(Pipeline="P24", Split=r["split"], Cohort=r["Cohort"],
                                 AUC=float(r["AUC"]), PRAUC=float(r["PRAUC"]), Brier=float(r["Brier"]),
                                 Acc=float(r["Acc"]), Precision=float(r["Precision"]), Recall=float(r["Recall"]),
                                 Thr=float(r["Thr"]), Cost=float(r["Cost"]), Notas="cost-5:1"))
    included_p24 = "summary"

print("P24:", included_p24, "| filas:", len(rows_p24))


P24: summary | filas: 10


In [10]:
rows = rows_p19 + rows_p22 + rows_p23 + rows_p24
master = pd.DataFrame(rows)

# Ordenar columnas y ordenar filas
col_order = ["Pipeline","Split","Cohort","AUC","PRAUC","Brier","Acc","Precision","Recall","Thr","Cost","Notas"]
for c in col_order:
    if c not in master.columns: master[c] = None
master = master[col_order]

master_path = OUT/"p25_master_table.csv"
master.to_csv(master_path, index=False)

print("✅ Guardado:", master_path)
print("Incluye → P19:", len(rows_p19), "| P22:", len(rows_p22), "| P23:", len(rows_p23), "| P24:", len(rows_p24))
display(master.sort_values(["Pipeline","Split","Cohort"]).reset_index(drop=True).head(20))


✅ Guardado: /content/drive/MyDrive/CognitivaAI/p25_informe_final/p25_master_table.csv
Incluye → P19: 6 | P22: 24 | P23: 8 | P24: 10


,Pipeline,Split,Cohort,AUC,PRAUC,Brier,Acc,Precision,Recall,Thr,Cost,Notas
0,P19,TEST,ALL,0.671464,0.606154,0.292286,NaN,NaN,NaN,NaN,NaN,XGBClassifier
1,P19,TEST,OAS1,0.662963,0.587607,0.309788,NaN,NaN,NaN,NaN,NaN,XGBClassifier
2,P19,TEST,OAS2,0.662879,0.682733,0.256521,NaN,NaN,NaN,NaN,NaN,XGBClassifier
3,P19,VAL,ALL,0.998302,0.998045,0.018682,NaN,NaN,NaN,NaN,NaN,XGBClassifier
4,P19,VAL,OAS1,0.998148,0.997619,0.017337,NaN,NaN,NaN,NaN,NaN,XGBClassifier
5,P19,VAL,OAS2,1.000000,1.000000,0.021553,NaN,NaN,NaN,NaN,NaN,XGBClassifier
6,P22,TEST,ALL,0.668174,0.646434,0.218939,NaN,NaN,NaN,NaN,NaN,LR_platt
7,P22,TEST,ALL,0.666941,0.617830,0.231128,NaN,NaN,NaN,NaN,NaN,LR_iso
8,P22,TEST,ALL,0.702303,0.628605,0.222275,NaN,NaN,NaN,NaN,NaN,HGB_platt
9,P22,TEST,ALL,0.665707,0.605231,0.239087,NaN,NaN,NaN,NaN,NaN,HGB_iso


In [11]:
from textwrap import dedent
import numpy as np

def fmt(x, d=3):
    try:
        if x is None or (isinstance(x,float) and (np.isnan(x))): return "-"
        return f"{float(x):.{d}f}"
    except:
        return str(x)

# Resumen rápido (TEST, por pipeline y cohorte, cuando exista)
def pipeline_glance(df, pipe, coh):
    q = df[(df.Pipeline==pipe) & (df.Split=="TEST") & (df.Cohort==coh)]
    if len(q)==0: return f"{pipe}-{coh}: n/a"
    r = q.iloc[0]
    return f"{pipe}-{coh}: AUC={fmt(r.AUC)} | PR-AUC={fmt(r.PRAUC)} | Brier={fmt(r.Brier)}"

glance_lines = []
for pipe in ["P19","P22","P23","P24"]:
    for coh in ["ALL","OAS1","OAS2"]:
        if len(master[(master.Pipeline==pipe) & (master.Split=="TEST") & (master.Cohort==coh)])>0:
            glance_lines.append(pipeline_glance(master, pipe, coh))

md_readme = dedent(f"""
### P25 — Informe final (consolidación)

**Tabla maestra:** `p25_informe_final/p25_master_table.csv`

**Resumen (TEST):**
{chr(10).join("- " + s for s in glance_lines)}

**Notas:**
- P24 (LR elastic-net + Platt) mantiene AUC≈{fmt(master[(master.Pipeline=='P24') & (master.Split=='TEST') & (master.Cohort=='ALL')]['AUC'].max())} global y **recupera señal en OAS2**.
- P23 aporta **umbrales coste-óptimos** por cohorte (FN:FP=5:1) útiles para decisión clínica.
- P19 (meta-XGB OOF) confirma techo de generalización similar al meta simple.
""").strip()

md_informe = dedent(f"""
## P25 — Consolidación y narrativa final

**Diseño:** unificación de resultados de P19, P22, P23 y P24 en una tabla maestra con métricas por cohorte (AUC, PR-AUC, Brier) y, cuando aplica, decisión por coste (Acc, Precision, Recall, Thr, Cost).

**Hallazgos clave (TEST):**
- {pipeline_glance(master,'P24','ALL')}
- {pipeline_glance(master,'P24','OAS1')}
- {pipeline_glance(master,'P24','OAS2')}
- {pipeline_glance(master,'P23','OAS1')} (decisión coste) · {pipeline_glance(master,'P23','OAS2')} (decisión coste)
- {pipeline_glance(master,'P19','ALL')}

**Operativo:** recomendamos aplicar los **umbrales P24 por cohorte** (coste 5:1) cuando el objetivo sea cribado (minimizar FN), manteniendo la calibración derivada de Platt.
""").strip()

from datetime import datetime
today = datetime.now().strftime("%Y-%m-%d")
md_bitacora = dedent(f"""
### {today} — P25 (construcción del informe final)

- Consolidé P19/P22/P23/P24 en `p25_master_table.csv`.
- Generé bloques de README/Informe/Bitácora con los números actuales.
- Próximo: añadir figuras (ROC/PR/Calibración) por cohorte y checklist de reproducibilidad.

""").strip()

print("\n--- README.md (bloque) ---\n")
print(md_readme)
print("\n--- InformeTecnico.md (bloque) ---\n")
print(md_informe)
print("\n--- CuadernoBitacora.md (bloque) ---\n")
print(md_bitacora)



--- README.md (bloque) ---

### P25 — Informe final (consolidación)

**Tabla maestra:** `p25_informe_final/p25_master_table.csv`

**Resumen (TEST):**
- P19-ALL: AUC=0.671 | PR-AUC=0.606 | Brier=0.292
- P19-OAS1: AUC=0.663 | PR-AUC=0.588 | Brier=0.310
- P19-OAS2: AUC=0.663 | PR-AUC=0.683 | Brier=0.257
- P22-ALL: AUC=0.668 | PR-AUC=0.646 | Brier=0.219
- P22-OAS1: AUC=0.756 | PR-AUC=0.726 | Brier=0.203
- P22-OAS2: AUC=0.504 | PR-AUC=0.524 | Brier=0.252
- P23-OAS1: AUC=0.743 | PR-AUC=0.657 | Brier=0.223
- P23-OAS2: AUC=0.500 | PR-AUC=0.522 | Brier=0.250
- P24-ALL: AUC=0.727 | PR-AUC=0.717 | Brier=0.220
- P24-OAS1: AUC=0.754 | PR-AUC=0.736 | Brier=0.211
- P24-OAS2: AUC=0.750 | PR-AUC=0.805 | Brier=0.238

**Notas:**
- P24 (LR elastic-net + Platt) mantiene AUC≈0.727 global y **recupera señal en OAS2**.
- P23 aporta **umbrales coste-óptimos** por cohorte (FN:FP=5:1) útiles para decisión clínica.
- P19 (meta-XGB OOF) confirma techo de generalización similar al meta simple.

--- InformeTecnico.

In [12]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, precision_recall_curve

FIG = OUT  # guarda aquí las figuras

def plot_curves_for_file(tag, path_csv, prob_col="y_prob"):
    df = load_csv(path_csv)
    df = add_cohort_col(df)
    ycol = detect_label_col(df); assert ycol and prob_col in df.columns
    for coh in ["ALL","OAS1","OAS2"]:
        part = df if coh=="ALL" else df[df['cohort']==coh]
        if len(part) < 3 or len(np.unique(part[ycol]))<2:
            continue
        y = part[ycol].astype(int).values
        p = part[prob_col].astype(float).values
        fpr, tpr, _ = roc_curve(y, p)
        pr, rc, _ = precision_recall_curve(y, p)

        plt.figure()
        plt.plot(fpr, tpr)
        plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(f"ROC {tag} – {coh}")
        plt.savefig(FIG/f"p25_roc_{tag}_{coh}.png", dpi=160, bbox_inches="tight"); plt.close()

        plt.figure()
        plt.plot(rc, pr)
        plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(f"PR {tag} – {coh}")
        plt.savefig(FIG/f"p25_pr_{tag}_{coh}.png", dpi=160, bbox_inches="tight"); plt.close()

# P19
if safe_exists(BASE/"p19_meta_ensemble"/"p19_val_patient_preds.csv"):
    plot_curves_for_file("P19_VAL", BASE/"p19_meta_ensemble"/"p19_val_patient_preds.csv")
if safe_exists(BASE/"p19_meta_ensemble"/"p19_test_patient_preds.csv"):
    plot_curves_for_file("P19_TEST", BASE/"p19_meta_ensemble"/"p19_test_patient_preds.csv")

# P24
if safe_exists(BASE/"p24_meta_simple"/"p24_val_preds.csv"):
    plot_curves_for_file("P24_VAL", BASE/"p24_meta_simple"/"p24_val_preds.csv")
if safe_exists(BASE/"p24_meta_simple"/"p24_test_preds.csv"):
    plot_curves_for_file("P24_TEST", BASE/"p24_meta_simple"/"p24_test_preds.csv")

print("Figuras guardadas en:", FIG)


Figuras guardadas en: /content/drive/MyDrive/CognitivaAI/p25_informe_final


In [13]:
from sklearn.calibration import calibration_curve

def plot_calibration(tag, path_csv, prob_col="y_prob", n_bins=8):
    df = load_csv(path_csv)
    df = add_cohort_col(df)
    ycol = detect_label_col(df); assert ycol and prob_col in df.columns
    for coh in ["ALL","OAS1","OAS2"]:
        part = df if coh=="ALL" else df[df['cohort']==coh]
        if len(part) < 15 or len(np.unique(part[ycol]))<2:
            continue
        y = part[ycol].astype(int).values
        p = part[prob_col].astype(float).values
        frac_pos, mean_pred = calibration_curve(y, p, n_bins=n_bins, strategy="quantile")
        plt.figure()
        plt.plot([0,1],[0,1], linestyle='--')
        plt.plot(mean_pred, frac_pos, marker='o')
        plt.xlabel("Predicted prob"); plt.ylabel("Observed freq")
        plt.title(f"Calibration {tag} – {coh}")
        plt.savefig(OUT/f"p25_cal_{tag}_{coh}.png", dpi=160, bbox_inches="tight"); plt.close()

# P19
if safe_exists(BASE/"p19_meta_ensemble"/"p19_val_patient_preds.csv"):
    plot_calibration("P19_VAL", BASE/"p19_meta_ensemble"/"p19_val_patient_preds.csv")
if safe_exists(BASE/"p19_meta_ensemble"/"p19_test_patient_preds.csv"):
    plot_calibration("P19_TEST", BASE/"p19_meta_ensemble"/"p19_test_patient_preds.csv")

# P24
if safe_exists(BASE/"p24_meta_simple"/"p24_val_preds.csv"):
    plot_calibration("P24_VAL", BASE/"p24_meta_simple"/"p24_val_preds.csv")
if safe_exists(BASE/"p24_meta_simple"/"p24_test_preds.csv"):
    plot_calibration("P24_TEST", BASE/"p24_meta_simple"/"p24_test_preds.csv")

print("Curvas de calibración guardadas en:", OUT)


Curvas de calibración guardadas en: /content/drive/MyDrive/CognitivaAI/p25_informe_final


In [14]:
# Coste(VAL) vs Umbral para P24 por cohorte, y barrido C_FN:C_FP ∈ {3,5,7,10}
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path

BASE = Path("/content/drive/MyDrive/CognitivaAI")
VAL = pd.read_csv(BASE/"p24_meta_simple"/"p24_val_preds.csv")  # y_prob calibrado
OUT = BASE/"p25_informe_final"

def cost_curve(df, C_FN=5.0, C_FP=1.0, n=501):
    y = df["y_true"].values.astype(int); p = df["y_prob"].values.astype(float)
    thr = np.linspace(0,1,n)
    cost, rec, prec = [], [], []
    for t in thr:
        yhat = (p>=t).astype(int)
        TP = ((yhat==1)&(y==1)).sum()
        FP = ((yhat==1)&(y==0)).sum()
        TN = ((yhat==0)&(y==0)).sum()
        FN = ((yhat==0)&(y==1)).sum()
        c = C_FN*FN + C_FP*FP
        cost.append(c)
        rec.append(TP/(TP+FN+1e-9))
        prec.append(TP/(TP+FP+1e-9))
    return pd.DataFrame({"thr":thr, "cost":cost, "recall":rec, "precision":prec})

for coh in ["OAS1","OAS2"]:
    part = VAL[VAL["cohort"]==coh]
    if len(part)==0: continue
    plt.figure()
    for C_FN in [3,5,7,10]:
        dfc = cost_curve(part, C_FN=C_FN, C_FP=1.0)
        plt.plot(dfc["thr"], dfc["cost"], label=f"{C_FN}:1")
    plt.xlabel("Umbral"); plt.ylabel("Coste(VAL)"); plt.title(f"Coste vs Umbral – {coh} (P24)")
    plt.legend()
    plt.savefig(OUT/f"p25_cost_curve_{coh}.png", dpi=160, bbox_inches="tight")
    plt.close()

print("Guardadas curvas de coste en:", OUT)


Guardadas curvas de coste en: /content/drive/MyDrive/CognitivaAI/p25_informe_final


In [15]:
import json, pandas as pd
from pathlib import Path

BASE = Path("/content/drive/MyDrive/CognitivaAI")
TEST = pd.read_csv(BASE/"p24_meta_simple"/"p24_test_preds.csv")
THR  = json.load(open(BASE/"p24_meta_simple"/"p24_thresholds.json"))

rows=[]
for coh in ["OAS1","OAS2"]:
    t = float(THR[coh]["thr"])
    part = TEST[TEST["cohort"]==coh].copy()
    y = part["y_true"].values; p = part["y_prob"].values
    yhat = (p>=t).astype(int)
    TP = int(((yhat==1)&(y==1)).sum())
    FP = int(((yhat==1)&(y==0)).sum())
    TN = int(((yhat==0)&(y==0)).sum())
    FN = int(((yhat==0)&(y==1)).sum())
    prec = TP/(TP+FP) if (TP+FP)>0 else 0.0
    rec  = TP/(TP+FN) if (TP+FN)>0 else 0.0
    acc  = (TP+TN)/(TP+TN+FP+FN)
    cost = 5.0*FN + 1.0*FP
    rows.append({"Cohort":coh, "Thr":t, "TP":TP,"FP":FP,"TN":TN,"FN":FN,
                 "Precision":prec,"Recall":rec,"Acc":acc,"Cost":cost})

conf = pd.DataFrame(rows)
conf.to_csv(BASE/"p25_informe_final"/"p25_confusion_test_p24.csv", index=False)
print(conf)
print("Guardado:", BASE/"p25_informe_final"/"p25_confusion_test_p24.csv")


  Cohort    Thr  TP  FP  TN  FN  Precision    Recall       Acc  Cost
0   OAS1  0.435  14   9  18   6   0.608696  0.700000  0.680851  39.0
1   OAS2  0.332  11   7   4   1   0.611111  0.916667  0.652174  12.0
Guardado: /content/drive/MyDrive/CognitivaAI/p25_informe_final/p25_confusion_test_p24.csv


In [16]:
import pandas as pd
from pathlib import Path
from textwrap import dedent

BASE = Path("/content/drive/MyDrive/CognitivaAI")
OUT  = BASE/"p25_informe_final"
master = pd.read_csv(OUT/"p25_master_table.csv")

# Nos quedamos con TEST y con filas clave (P19, P22 mejores, P23 coste, P24 Platt)
keep = master[(master["Split"]=="TEST") & master["Pipeline"].isin(["P19","P22","P23","P24"])].copy()

# Orden recomendado
order = pd.Categorical(keep["Pipeline"], categories=["P24","P23","P22","P19"], ordered=True)
keep = keep.assign(PipelineOrder=order).sort_values(["PipelineOrder","Cohort","Notas"])

cols = ["Pipeline","Cohort","Notas","AUC","PRAUC","Brier","Acc","Precision","Recall","Thr","Cost"]
keep = keep[cols]

# Markdown
def fmt(x, d=3):
    try: return f"{float(x):.{d}f}"
    except: return "-" if pd.isna(x) else str(x)

md = ["| Pipeline | Cohorte | Método | AUC | PR-AUC | Brier | Acc | Prec | Rec | Thr | Coste |",
      "|---|---|---:|---:|---:|---:|---:|---:|---:|---:|---:|"]
for _,r in keep.iterrows():
    md.append(f"| {r.Pipeline} | {r.Cohort} | {r.Notas} | {fmt(r.AUC)} | {fmt(r.PRAUC)} | {fmt(r.Brier)} | {fmt(r.Acc)} | {fmt(r.Precision)} | {fmt(r.Recall)} | {fmt(r.Thr)} | {fmt(r.Cost)} |")

md_text = "\n".join(md)
(Path(OUT/"p25_executive_table.md")).write_text(md_text, encoding="utf-8")
print(md_text)
print("Guardado:", OUT/"p25_executive_table.md")


| Pipeline | Cohorte | Método | AUC | PR-AUC | Brier | Acc | Prec | Rec | Thr | Coste |
|---|---|---:|---:|---:|---:|---:|---:|---:|---:|---:|
| P24 | ALL | Platt | 0.727 | 0.717 | 0.220 | nan | nan | nan | nan | nan |
| P24 | OAS1 | Platt | 0.754 | 0.736 | 0.211 | nan | nan | nan | nan | nan |
| P24 | OAS1 | cost-5:1 | 0.754 | 0.736 | 0.211 | 0.681 | 0.609 | 0.700 | 0.435 | 39.000 |
| P24 | OAS2 | Platt | 0.750 | 0.805 | 0.238 | nan | nan | nan | nan | nan |
| P24 | OAS2 | cost-5:1 | 0.750 | 0.805 | 0.238 | 0.652 | 0.611 | 0.917 | 0.332 | 12.000 |
| P23 | OAS1 | isotonic | 0.743 | 0.657 | 0.223 | 0.574 | 0.500 | 0.950 | nan | 24.000 |
| P23 | OAS1 | platt | 0.724 | 0.649 | 0.210 | 0.574 | 0.500 | 0.950 | nan | 24.000 |
| P23 | OAS2 | isotonic | 0.500 | 0.522 | 0.250 | 0.522 | 0.522 | 1.000 | nan | 11.000 |
| P23 | OAS2 | platt | 0.500 | 0.522 | 0.250 | 0.522 | 0.522 | 1.000 | nan | 11.000 |
| P22 | ALL | HGB_iso | 0.666 | 0.605 | 0.239 | nan | nan | nan | nan | nan |
| P22 | ALL | HGB

In [17]:
import hashlib, json, os, sys, platform, subprocess
from pathlib import Path

BASE = Path("/content/drive/MyDrive/CognitivaAI")
REL  = BASE/"p25_release"
REL.mkdir(parents=True, exist_ok=True)

# 1) Copia artefactos clave
to_copy = [
    # P19
    BASE/"p19_meta_ensemble"/"p19_summary.json",
    BASE/"p19_meta_ensemble"/"p19_val_patient_preds.csv",
    BASE/"p19_meta_ensemble"/"p19_test_patient_preds.csv",
    # P23
    BASE/"p23_meta_costcohort"/"p23_summary.json",
    BASE/"p23_meta_costcohort"/"p23_thresholds.json",
    # P24
    BASE/"p24_meta_simple"/"p24_summary.json",
    BASE/"p24_meta_simple"/"p24_val_preds.csv",
    BASE/"p24_meta_simple"/"p24_test_preds.csv",
    BASE/"p24_meta_simple"/"p24_thresholds.json",
    BASE/"p24_meta_simple"/"p24_coefficients.csv",
    BASE/"p24_meta_simple"/"p24_model.pkl",
    BASE/"p24_meta_simple"/"p24_platt.pkl",
    # P25
    BASE/"p25_informe_final"/"p25_master_table.csv",
    BASE/"p25_informe_final"/"p25_executive_table.md",
    BASE/"p25_informe_final"/"p25_confusion_test_p24.csv",
    # Figuras
    BASE/"p25_informe_final"/"p25_roc_P24_TEST_OAS1.png",
    BASE/"p25_informe_final"/"p25_roc_P24_TEST_OAS2.png",
    BASE/"p25_informe_final"/"p25_pr_P24_TEST_OAS1.png",
    BASE/"p25_informe_final"/"p25_pr_P24_TEST_OAS2.png",
    BASE/"p25_informe_final"/"p25_cal_P24_TEST_OAS1.png",
    BASE/"p25_informe_final"/"p25_cal_P24_TEST_OAS2.png",
    BASE/"p25_informe_final"/"p25_cost_curve_OAS1.png",
    BASE/"p25_informe_final"/"p25_cost_curve_OAS2.png",
]
for p in to_copy:
    try:
        if p.exists():
            dest = REL/p.name
            dest.write_bytes(p.read_bytes())
    except Exception as e:
        print("[WARN] No copié", p, "→", e)

# 2) Hashes SHA256 de todo el release
def sha256_of(path):
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(1<<20), b""):
            h.update(chunk)
    return h.hexdigest()

manifest=[]
for p in sorted(REL.iterdir()):
    if p.is_file():
        manifest.append({"file": p.name, "sha256": sha256_of(p)})
(Path(REL/"MANIFEST.json")).write_text(json.dumps(manifest, indent=2), encoding="utf-8")

# 3) Entorno (versiones críticas)
env = {
    "python": sys.version,
    "platform": platform.platform(),
}
def pkg_ver(pkg):
    try:
        import importlib, pkg_resources
        m = importlib.import_module(pkg)
        v = getattr(m, "__version__", None) or pkg_resources.get_distribution(pkg).version
        return v
    except Exception:
        return None

for pkg in ["numpy","pandas","scikit-learn","xgboost","lightgbm"]:
    env[pkg] = pkg_ver(pkg)

(Path(REL/"ENVIRONMENT.json")).write_text(json.dumps(env, indent=2), encoding="utf-8")

print("📦 Release listo en:", REL)
print("Archivos:", [p.name for p in REL.iterdir()])


/tmp/ipython-input-2900077532.py:68: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import importlib, pkg_resources


📦 Release listo en: /content/drive/MyDrive/CognitivaAI/p25_release
Archivos: ['p19_summary.json', 'p19_val_patient_preds.csv', 'p19_test_patient_preds.csv', 'p23_summary.json', 'p23_thresholds.json', 'p24_summary.json', 'p24_val_preds.csv', 'p24_test_preds.csv', 'p24_thresholds.json', 'p24_coefficients.csv', 'p24_model.pkl', 'p24_platt.pkl', 'p25_master_table.csv', 'p25_executive_table.md', 'p25_confusion_test_p24.csv', 'p25_roc_P24_TEST_OAS1.png', 'p25_roc_P24_TEST_OAS2.png', 'p25_pr_P24_TEST_OAS1.png', 'p25_pr_P24_TEST_OAS2.png', 'p25_cal_P24_TEST_OAS1.png', 'p25_cal_P24_TEST_OAS2.png', 'p25_cost_curve_OAS1.png', 'p25_cost_curve_OAS2.png', 'MANIFEST.json', 'ENVIRONMENT.json']


In [18]:
from pathlib import Path
card = f"""
# Model Card — CognitivaAI P24 (LR elastic-net + Platt)

**Fecha:** 2025-09-07
**Versión:** P24 (meta simple) · **Calibración:** Platt · **Umbrales (FN:FP=5:1):** OAS1=0.435, OAS2=0.332

## 1. Descripción
Clasificador binario (Control=0 vs Dementia/Converted=1) entrenado con OASIS-1 (p11) y OASIS-2 (p14) a nivel **paciente**, a partir de agregaciones de 20 slices axiales por volumen. Meta sencillo **LR elastic-net** con **calibración Platt**.

## 2. Datos
- **OASIS-1** (principal hasta p18), **OASIS-2** (1 visita/paciente, desde p19).
- Prepro: 20 slices equiespaciados, z-score, CLAHE opcional.
- Splits estratificados a nivel **paciente**.

## 3. Entrenamiento y validación
- RepeatedStratifiedKFold 5×5; búsqueda de {{"clf__C": 0.1, "clf__l1_ratio": 0.7}}.
- Calibración **Platt** en VAL.
- Umbrales coste-óptimos (VAL) por cohorte con **C_FN: C_FP = 5:1**.

## 4. Métricas (TEST)
- **Global:** AUC=0.727, PR-AUC=0.717, Brier=0.220.
- **OAS1:** AUC=0.754, PR-AUC=0.736, Brier=0.211.
- **OAS2:** AUC=0.750, PR-AUC=0.805, Brier=0.238.

## 5. Decisión (TEST @ umbral por cohorte)
- **OAS1 thr=0.435:** TP=14, FP=9, TN=18, FN=6 → Recall=0.70, Precision=0.61, Acc=0.681, Coste=39.
- **OAS2 thr=0.332:** TP=11, FP=7, TN=4, FN=1 → Recall=0.917, Precision=0.611, Acc=0.652, Coste=12.

## 6. Entrada/Salida
- **Input:** vector de 56 features por paciente (agregadas por slice).
- **Output:** probabilidad calibrada de clase positiva + decisión por cohorte según umbral.

## 7. Uso recomendado
- **Cribado** (minimizar FN): aplicar **umbrales P24** por cohorte.
- Reportar probabilidades calibradas cuando se precise interpretación.

## 8. Limitaciones
- Tamaño muestral reducido (riesgo de varianza/intervalos amplios).
- Desbalance y **shift** entre OAS1/OAS2; se mitiga con calibración y umbrales por cohorte.
- No validado externamente fuera de OASIS.

## 9. Consideraciones éticas
- Probabilidades y decisiones **no sustituyen** juicio clínico.
- Revisar sesgos por cohorte/centro; usar en contexto de soporte a la decisión.

## 10. Artefactos
Ver carpeta `p25_release/` (MANIFEST.json con hashes; ENVIRONMENT.json con versiones).
"""
out = Path("/content/drive/MyDrive/CognitivaAI/p25_release/MODEL_CARD.md")
out.write_text(card.strip(), encoding="utf-8")
print("Guardado:", out)


Guardado: /content/drive/MyDrive/CognitivaAI/p25_release/MODEL_CARD.md


In [22]:
# === Inferencia P24 (LR elastic-net + Platt) con features reales de test ===
import pickle, json, numpy as np, pandas as pd
from pathlib import Path
from glob import glob

BASE = Path("/content/drive/MyDrive/CognitivaAI")
P24  = BASE/"p24_meta_simple"

# 1) Carga modelo, calibrador y umbrales
model = pickle.load(open(P24/"p24_model.pkl","rb"))   # pipeline con imputer+scaler+LR
platt = pickle.load(open(P24/"p24_platt.pkl","rb"))   # calibrador Platt (entrenado en VAL)
thr   = json.load(open(P24/"p24_thresholds.json","r"))

# 2) Recupera lista y orden de features desde coeficientes (garantiza match con entrenamiento)
coef_df = pd.read_csv(P24/"p24_coefficients.csv")
feat_cols = coef_df["feature"].tolist()
assert len(feat_cols) == 56, f"Esperaba 56 features, tengo {len(feat_cols)}"

# 3) Carga y fusiona features por paciente (TEST). Añade aquí más fuentes si las tienes
def _clean_cols(df):
    df.columns = [str(c).replace("\ufeff","").strip() for c in df.columns]
    return df

def load_features(files):
    acc = None
    for f in files:
        if not Path(f).exists():
            continue
        df = _clean_cols(pd.read_csv(f, encoding="utf-8-sig"))
        # localizar columna id
        idcol = None
        for c in df.columns:
            if c.lower() in ["patient_id","patientid","pid","patient"]:
                idcol = c; break
        if idcol is None:
            continue
        df = df.rename(columns={idcol:"patient_id"})
        # solo numéricas + id
        numcols = ["patient_id"] + [c for c in df.columns if c!="patient_id" and pd.api.types.is_numeric_dtype(df[c])]
        df = df[numcols].drop_duplicates("patient_id")
        acc = df if acc is None else acc.merge(df, on="patient_id", how="outer")
    if acc is None:
        raise FileNotFoundError("No encontré CSVs de features. Revisa rutas.")
    acc["patient_id"] = acc["patient_id"].astype(str).str.replace("\ufeff","", regex=False).str.strip()
    return acc

feature_files = [
    BASE/"p11_alt_backbones/test_patient_features_backbones.csv",
    # Intenta traer también las features de OASIS-2 si existen:
    *[Path(p) for p in glob(str(BASE/"p14_oasis2_images/*patient_features*.csv"))],
]
feat_all = load_features(feature_files)

# 4) Construye matriz X alineada a feat_cols; samplea 1 paciente válido
X_all = feat_all.set_index("patient_id")
# asegura tener todas las columnas (faltantes se imputan dentro del pipeline)
for c in feat_cols:
    if c not in X_all.columns:
        X_all[c] = np.nan
X_all = X_all[feat_cols].sort_index()

# Elige un paciente real (mira algunos ids)
print("Ejemplos de patient_id disponibles:", list(X_all.index[:5]))
patient_id = list(X_all.index[:1])[0]  # <-- cambia aquí si quieres otro
print("Usando patient_id:", patient_id)

X_patient = X_all.loc[[patient_id]]

# 5) Probabilidad cruda del modelo y calibración Platt (maneja ambos casos: Platt sobre scores o sobre X)
proba_raw = model.predict_proba(X_patient.values)[:,1]

def apply_platt(platt, proba_raw, X_patient):
    # si el calibrador fue entrenado sobre 'scores' 1D (np.array), aceptará (n,1)
    if hasattr(platt, "predict_proba"):
        try:
            return platt.predict_proba(proba_raw.reshape(-1,1))[:,1]
        except Exception:
            # si es CalibratedClassifierCV sobre features, úsalo con X_patient
            return platt.predict_proba(X_patient.values)[:,1]
    # fallback: sin calibración
    return proba_raw

proba_cal = apply_platt(platt, proba_raw, X_patient)

# 6) Cohorte a partir del patient_id y decisión por umbral
def infer_cohort(pid):
    s = str(pid).strip().upper()
    return "OAS1" if s.startswith("OAS1") else ("OAS2" if s.startswith("OAS2") else "OAS1")

cohort = infer_cohort(patient_id)
t = float(thr[cohort]["thr"])
pred = int(proba_cal[0] >= t)

print({
    "patient_id": patient_id,
    "cohort": cohort,
    "proba_raw": float(proba_raw[0]),
    "proba_cal": float(proba_cal[0]),
    "thr": t,
    "y_pred": pred
})


def infer_cohort(pid):
    s = str(pid).strip().upper()
    return "OAS1" if s.startswith("OAS1") else ("OAS2" if s.startswith("OAS2") else "OAS1")

def predict_patient(pid):
    X = X_all.loc[[pid]][feat_cols].values  # NumPy para evitar warnings
    p_raw = model.predict_proba(X)[:,1]
    try:
        p_cal = platt.predict_proba(p_raw.reshape(-1,1))[:,1]  # Platt sobre scores
    except Exception:
        p_cal = platt.predict_proba(X)[:,1]                    # Fallback: Platt sobre X
    coh = infer_cohort(pid)
    t = float(thr[coh]["thr"])
    return {
        "patient_id": pid,
        "cohort": coh,
        "proba_raw": float(p_raw[0]),
        "proba_cal": float(p_cal[0]),
        "thr": t,
        "y_pred": int(p_cal[0] >= t),
    }

# Ejemplo 1 paciente (el mismo que usaste)
print(predict_patient("OAS1_0002"))

# Batch en todos los pacientes que tengas en X_all
preds = [predict_patient(pid) for pid in X_all.index]
preds_df = pd.DataFrame(preds)

# (Opcional) añadir y_true para evaluar rápidamente si tienes el CSV de test de p22
try:
    labels = pd.read_csv(BASE/"p22_meta_ablation"/"p22_test_calibrations.csv", encoding="utf-8-sig")
    idcol = next(c for c in labels.columns if c.lower() in ["patient_id","patientid","pid","patient"])
    ycol  = next(c for c in labels.columns if c.lower() in ["y_true","y","target","label","truth","gt"])
    labels = labels.rename(columns={idcol:"patient_id"})[["patient_id", ycol]].rename(columns={ycol:"y_true"})
    preds_df = preds_df.merge(labels, on="patient_id", how="left")
except Exception:
    pass

out_path = BASE/"p25_informe_final"/"p25_inference_demo.csv"
preds_df.to_csv(out_path, index=False)
print("Guardado:", out_path)
preds_df.head()



Ejemplos de patient_id disponibles: ['OAS1_0002', 'OAS1_0023', 'OAS1_0070', 'OAS1_0074', 'OAS1_0094']
Usando patient_id: OAS1_0002
{'patient_id': 'OAS1_0002', 'cohort': 'OAS1', 'proba_raw': 0.39292730824001604, 'proba_cal': 0.40765948801369756, 'thr': 0.435, 'y_pred': 0}
{'patient_id': 'OAS1_0002', 'cohort': 'OAS1', 'proba_raw': 0.39292730824001604, 'proba_cal': 0.40765948801369756, 'thr': 0.435, 'y_pred': 0}
Guardado: /content/drive/MyDrive/CognitivaAI/p25_informe_final/p25_inference_demo.csv


,patient_id,cohort,proba_raw,proba_cal,thr,y_pred,y_true
0,OAS1_0002,OAS1,0.392927,0.407659,0.435,0,0.0
1,OAS1_0023,OAS1,0.656631,0.591442,0.435,1,1.0
2,OAS1_0070,OAS1,0.411698,0.420500,0.435,0,0.0
3,OAS1_0074,OAS1,0.385908,0.402889,0.435,0,0.0
4,OAS1_0094,OAS1,0.411545,0.420395,0.435,0,1.0


In [25]:
# Verificación end-to-end (métricas + confusión) sobre p25_inference_demo.csv (robusta a y_true NaN)
import json, numpy as np, pandas as pd
from pathlib import Path
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss

BASE = Path("/content/drive/MyDrive/CognitivaAI")
demo = pd.read_csv(BASE/"p25_informe_final"/"p25_inference_demo.csv")
thr  = json.load(open(BASE/"p24_meta_simple"/"p24_thresholds.json"))

# ---- Limpieza y chequeos ----
# Asegura tipos numéricos
demo["proba_cal"] = pd.to_numeric(demo["proba_cal"], errors="coerce")
# y_true puede venir como float 0.0/1.0 o NaN; mantenemos NaN para filtrar
if "y_true" in demo.columns:
    demo["y_true"] = pd.to_numeric(demo["y_true"], errors="coerce")

# Report de faltantes
n_total = len(demo)
n_missing = demo["y_true"].isna().sum() if "y_true" in demo.columns else n_total
print(f"Total filas demo: {n_total} | con y_true NaN: {n_missing}")

# Filtra solo filas con etiqueta y probabilidad válidas
valid = demo.dropna(subset=["y_true","proba_cal"]).copy()
valid["y_true"] = valid["y_true"].astype(int)

print("Filas válidas por cohorte:")
print(valid.groupby("cohort")["y_true"].agg(["count","sum"]))

def metrics(y, p):
    y = np.asarray(y).astype(int)
    p = np.asarray(p).astype(float)
    has_var = (np.unique(y).size > 1)
    return dict(
        AUC   = float(roc_auc_score(y, p)) if has_var else float("nan"),
        PRAUC = float(average_precision_score(y, p)) if has_var else float("nan"),
        Brier = float(brier_score_loss(y, p))
    )

rows = []

# Global (si hay algo válido)
if len(valid) > 1 and valid["y_true"].nunique() > 1:
    rows.append(dict(Split="TEST", Cohort="ALL", **metrics(valid["y_true"], valid["proba_cal"])))

# Por cohorte + confusión a umbral
for coh in ["OAS1","OAS2"]:
    part = valid[valid["cohort"]==coh]
    if len(part) == 0:
        continue
    # Métricas de probabilidad (requieren ambas clases)
    m = metrics(part["y_true"], part["proba_cal"]) if part["y_true"].nunique() > 1 else {"AUC":float("nan"),"PRAUC":float("nan"),"Brier":brier_score_loss(part["y_true"], part["proba_cal"])}
    # Confusión a umbral (solo tiene sentido si hay positivos y negativos)
    t = float(thr[coh]["thr"])
    y = part["y_true"].values
    yhat = (part["proba_cal"].values >= t).astype(int)
    TP = int(((yhat==1)&(y==1)).sum())
    FP = int(((yhat==1)&(y==0)).sum())
    TN = int(((yhat==0)&(y==0)).sum())
    FN = int(((yhat==0)&(y==1)).sum())
    denom_p = (TP+FP)
    denom_r = (TP+FN)
    prec = TP/denom_p if denom_p>0 else float("nan")
    rec  = TP/denom_r if denom_r>0 else float("nan")
    acc  = (TP+TN)/(TP+TN+FP+FN) if (TP+TN+FP+FN)>0 else float("nan")
    cost = 5.0*FN + 1.0*FP

    rows.append(dict(Split="TEST", Cohort=coh, **m, Thr=t, Acc=acc, Precision=prec, Recall=rec, Cost=cost,
                     TP=TP, FP=FP, TN=TN, FN=FN))

chk = pd.DataFrame(rows)
print("\nResumen verificación (TEST):")
print(chk)

# (Opcional) guarda el resumen
out_path = BASE/"p25_informe_final"/"p25_inference_demo_eval.csv"
chk.to_csv(out_path, index=False)
print("Guardado:", out_path)

# (Opcional) lista de patient_id sin y_true para depurar
if n_missing > 0:
    missing_ids = demo.loc[demo["y_true"].isna(), "patient_id"].tolist()
    print("\nPatient_id sin etiqueta (muestra hasta 10):", missing_ids[:10])



Total filas demo: 92 | con y_true NaN: 22
Filas válidas por cohorte:
        count  sum
cohort            
OAS1       47   20
OAS2       23   12

Resumen verificación (TEST):
  Split Cohort       AUC     PRAUC     Brier    Thr       Acc  Precision  \
0  TEST    ALL  0.726974  0.716780  0.219878    NaN       NaN        NaN   
1  TEST   OAS1  0.753704  0.735858  0.210779  0.435  0.680851   0.608696   
2  TEST   OAS2  0.750000  0.804859  0.238472  0.332  0.652174   0.611111   

     Recall  Cost    TP   FP    TN   FN  
0       NaN   NaN   NaN  NaN   NaN  NaN  
1  0.700000  39.0  14.0  9.0  18.0  6.0  
2  0.916667  12.0  11.0  7.0   4.0  1.0  
Guardado: /content/drive/MyDrive/CognitivaAI/p25_informe_final/p25_inference_demo_eval.csv

Patient_id sin etiqueta (muestra hasta 10): ['OAS2_0002', 'OAS2_0004', 'OAS2_0005', 'OAS2_0020', 'OAS2_0021', 'OAS2_0026', 'OAS2_0034', 'OAS2_0037', 'OAS2_0042', 'OAS2_0053']


In [26]:
# Comprueba que proba_cal del demo coincide con y_prob de P24 (mismas filas de TEST)
import numpy as np, pandas as pd
from pathlib import Path

BASE = Path("/content/drive/MyDrive/CognitivaAI")
demo = pd.read_csv(BASE/"p25_informe_final"/"p25_inference_demo.csv")
p24t = pd.read_csv(BASE/"p24_meta_simple"/"p24_test_preds.csv")  # contiene y_prob (P24 TEST)

# Merge por patient_id y calcula discrepancias
q = demo.merge(p24t[["patient_id","y_prob"]], on="patient_id", how="inner")
q["abs_err"] = (q["proba_cal"] - q["y_prob"]).abs()
mae = q["abs_err"].mean()
mx  = q["abs_err"].max()
corr = np.corrcoef(q["proba_cal"], q["y_prob"])[0,1] if len(q)>1 else np.nan

print(f"Coincidencias: {len(q)}/{len(demo)}")
print(f"MAE = {mae:.6f} | max|err| = {mx:.6f} | corr = {corr:.6f}")
# Muestra los peores 5 casos (si existieran)
print(q.sort_values("abs_err", ascending=False).head(5)[["patient_id","proba_cal","y_prob","abs_err"]])


Coincidencias: 70/92
MAE = 0.000000 | max|err| = 0.000000 | corr = 1.000000
   patient_id  proba_cal    y_prob       abs_err
23  OAS1_0290   0.692370  0.692370  1.110223e-16
24  OAS1_0291   0.356955  0.356955  1.110223e-16
2   OAS1_0070   0.420500  0.420500  0.000000e+00
3   OAS1_0074   0.402889  0.402889  0.000000e+00
0   OAS1_0002   0.407659  0.407659  0.000000e+00


In [27]:
import pandas as pd
from pathlib import Path

BASE = Path("/content/drive/MyDrive/CognitivaAI")
demo = pd.read_csv(BASE/"p25_informe_final"/"p25_inference_demo.csv")

with_label    = demo.dropna(subset=["y_true"]).copy()
without_label = demo[demo["y_true"].isna()].drop(columns=["y_true"], errors="ignore").copy()

with_label.to_csv(BASE/"p25_informe_final"/"p25_predictions_labeled.csv", index=False)
without_label.to_csv(BASE/"p25_informe_final"/"p25_predictions_unlabeled.csv", index=False)

print("Guardado:", BASE/"p25_informe_final"/"p25_predictions_labeled.csv")
print("Guardado:", BASE/"p25_informe_final"/"p25_predictions_unlabeled.csv")
print("Resumen labeled:", with_label.shape, "Resumen unlabeled:", without_label.shape)


Guardado: /content/drive/MyDrive/CognitivaAI/p25_informe_final/p25_predictions_labeled.csv
Guardado: /content/drive/MyDrive/CognitivaAI/p25_informe_final/p25_predictions_unlabeled.csv
Resumen labeled: (70, 7) Resumen unlabeled: (22, 6)


In [28]:
import pandas as pd
from pathlib import Path
from textwrap import dedent

BASE = Path("/content/drive/MyDrive/CognitivaAI")
coef = pd.read_csv(BASE/"p24_meta_simple"/"p24_coefficients.csv")
coef["abs_coef"] = coef["coef"].abs()
top = coef.sort_values("abs_coef", ascending=False).head(15)
top.to_csv(BASE/"p25_informe_final"/"p25_p24_top_coefficients.csv", index=False)

md = ["| Rank | Feature | Coef | |Coef| |", "|---:|---|---:|---:|"]
for i, r in enumerate(top.itertuples(index=False), 1):
    md.append(f"| {i} | {r.feature} | {r.coef:.4f} | {r.abs_coef:.4f} |")
md_text = "\n".join(md)

(Path(BASE/"p25_informe_final"/"p25_p24_top_coefficients.md")).write_text(md_text, encoding="utf-8")
print(md_text)
print("Guardado:", BASE/"p25_informe_final"/"p25_p24_top_coefficients.md")


| Rank | Feature | Coef | |Coef| |
|---:|---|---:|---:|
| 1 | oas2_effb3_p14_mean | 0.1528 | 0.1528 |
| 2 | oas2_effb3_p14_trimmed20 | 0.1409 | 0.1409 |
| 3 | slice_preds_plus_top7 | 0.1330 | 0.1330 |
| 4 | slice_preds_plus_p2 | 0.1218 | 0.1218 |
| 5 | slice_preds_plus_mean | 0.1200 | 0.1200 |
| 6 | patient_preds_plus_p2 | 0.0888 | 0.0888 |
| 7 | patient_preds_plus_mean | 0.0888 | 0.0888 |
| 8 | patient_preds_plus_trimmed20 | 0.0888 | 0.0888 |
| 9 | patient_preds_plus_top7 | 0.0888 | 0.0888 |
| 10 | oas2_effb3_mean | 0.0726 | 0.0726 |
| 11 | slice_preds_plus_trimmed20 | 0.0333 | 0.0333 |
| 12 | oas2_effb3_trimmed20 | 0.0156 | 0.0156 |
| 13 | oas2_effb3_p14_p2 | 0.0107 | 0.0107 |
| 14 | oas2_effb3_p14_top7 | 0.0000 | 0.0000 |
| 15 | oas2_effb3_p2 | 0.0000 | 0.0000 |
Guardado: /content/drive/MyDrive/CognitivaAI/p25_informe_final/p25_p24_top_coefficients.md


In [29]:
from textwrap import dedent
block = dedent("""
**Modelo final sugerido (P25):** P24 (LR elastic-net + Platt) con umbrales por cohorte (FN:FP=5:1): **OAS1=0.435**, **OAS2=0.332**.
- TEST @ OAS1: TP=14, FP=9, TN=18, FN=6 → Recall=0.70, Precision=0.61, Acc=0.681, Coste=39.
- TEST @ OAS2: TP=11, FP=7, TN=4, FN=1 → Recall=0.917, Precision=0.611, Acc=0.652, Coste=12.
Release: `p25_release/` (MANIFEST.json, ENVIRONMENT.json, MODEL_CARD.md).
""").strip()
print(block)


**Modelo final sugerido (P25):** P24 (LR elastic-net + Platt) con umbrales por cohorte (FN:FP=5:1): **OAS1=0.435**, **OAS2=0.332**.
- TEST @ OAS1: TP=14, FP=9, TN=18, FN=6 → Recall=0.70, Precision=0.61, Acc=0.681, Coste=39.
- TEST @ OAS2: TP=11, FP=7, TN=4, FN=1 → Recall=0.917, Precision=0.611, Acc=0.652, Coste=12.
Release: `p25_release/` (MANIFEST.json, ENVIRONMENT.json, MODEL_CARD.md).


In [30]:
# Sensibilidad coste: para C_FN:C_FP ∈ {3,5,7,10} → umbral en VAL y evaluación en TEST (P24)
import numpy as np, pandas as pd
from pathlib import Path

BASE = Path("/content/drive/MyDrive/CognitivaAI")
VAL  = pd.read_csv(BASE/"p24_meta_simple"/"p24_val_preds.csv")
TEST = pd.read_csv(BASE/"p24_meta_simple"/"p24_test_preds.csv")

def choose_thr_cost(df, C_FN=5.0, C_FP=1.0, n=1001):
    y = df["y_true"].astype(int).values
    p = df["y_prob"].astype(float).values
    thr = np.linspace(0,1,n)
    best = None
    for t in thr:
        yhat = (p>=t).astype(int)
        FP = int(((yhat==1)&(y==0)).sum())
        FN = int(((yhat==0)&(y==1)).sum())
        cost = C_FN*FN + C_FP*FP
        if best is None or cost < best[0]:
            best = (cost, t, FP, FN)
    return {"thr":best[1], "cost_val":best[0], "FP_val":best[2], "FN_val":best[3]}

def eval_at_thr(df, t, C_FN=5.0, C_FP=1.0):
    y = df["y_true"].astype(int).values
    p = df["y_prob"].astype(float).values
    yhat = (p>=t).astype(int)
    TP = int(((yhat==1)&(y==1)).sum()); FP = int(((yhat==1)&(y==0)).sum())
    TN = int(((yhat==0)&(y==0)).sum()); FN = int(((yhat==0)&(y==1)).sum())
    prec = TP/(TP+FP) if (TP+FP)>0 else np.nan
    rec  = TP/(TP+FN) if (TP+FN)>0 else np.nan
    acc  = (TP+TN)/(TP+TN+FP+FN) if (TP+TN+FP+FN)>0 else np.nan
    cost = C_FN*FN + C_FP*FP
    return {"TP":TP,"FP":FP,"TN":TN,"FN":FN,"Precision":prec,"Recall":rec,"Acc":acc,"Cost":cost}

rows=[]
for coh in ["OAS1","OAS2"]:
    v = VAL[VAL["cohort"]==coh]
    t = TEST[TEST["cohort"]==coh]
    for C_FN in [3,5,7,10]:
        sel = choose_thr_cost(v, C_FN=C_FN, C_FP=1.0)
        ev  = eval_at_thr(t, sel["thr"], C_FN=C_FN, C_FP=1.0)
        rows.append({"Cohort":coh,"C_FN":C_FN,"thr_val":sel["thr"],"cost_val":sel["cost_val"], **ev})

sens = pd.DataFrame(rows)
out = BASE/"p25_informe_final"/"p25_cost_sensitivity_p24.csv"
sens.to_csv(out, index=False)
print("Guardado:", out)
sens


Guardado: /content/drive/MyDrive/CognitivaAI/p25_informe_final/p25_cost_sensitivity_p24.csv


,Cohort,C_FN,thr_val,cost_val,TP,FP,TN,FN,Precision,Recall,Acc,Cost
0,OAS1,3,0.435,9.0,14,9,18,6,0.608696,0.700000,0.680851,27.0
1,OAS1,5,0.435,11.0,14,9,18,6,0.608696,0.700000,0.680851,39.0
2,OAS1,7,0.435,13.0,14,9,18,6,0.608696,0.700000,0.680851,51.0
3,OAS1,10,0.435,16.0,14,9,18,6,0.608696,0.700000,0.680851,69.0
4,OAS2,3,0.332,6.0,11,7,4,1,0.611111,0.916667,0.652174,10.0
5,OAS2,5,0.332,6.0,11,7,4,1,0.611111,0.916667,0.652174,12.0
6,OAS2,7,0.332,6.0,11,7,4,1,0.611111,0.916667,0.652174,14.0
7,OAS2,10,0.332,6.0,11,7,4,1,0.611111,0.916667,0.652174,17.0


In [31]:
# Bootstrap 2000 repeticiones (rápido en Colab). Cambia B si quieres más.
import numpy as np, pandas as pd
from pathlib import Path
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss

BASE = Path("/content/drive/MyDrive/CognitivaAI")
TEST = pd.read_csv(BASE/"p24_meta_simple"/"p24_test_preds.csv")

def bootstrap_ci(y, p, B=2000, seed=42):
    rng = np.random.default_rng(seed)
    n = len(y)
    aucs, prs, brs = [], [], []
    for _ in range(B):
        idx = rng.integers(0, n, n)
        yy, pp = y[idx], p[idx]
        if len(np.unique(yy))>1:
            aucs.append(roc_auc_score(yy, pp))
            prs.append(average_precision_score(yy, pp))
        brs.append(brier_score_loss(yy, pp))
    def ci(a):
        if len(a)==0: return (np.nan,np.nan,np.nan)
        a = np.sort(a); lo=a[int(0.025*len(a))]; md=a[int(0.5*len(a))]; hi=a[int(0.975*len(a))]
        return (md, lo, hi)
    return {"AUC": ci(aucs), "PRAUC": ci(prs), "Brier": ci(brs)}

rows=[]
for coh in ["ALL","OAS1","OAS2"]:
    part = TEST if coh=="ALL" else TEST[TEST["cohort"]==coh]
    y = part["y_true"].astype(int).values
    p = part["y_prob"].astype(float).values
    res = bootstrap_ci(y,p,B=2000,seed=42)
    rows.append({
        "Cohort":coh,
        "AUC_med":res["AUC"][0], "AUC_lo":res["AUC"][1], "AUC_hi":res["AUC"][2],
        "PRAUC_med":res["PRAUC"][0], "PRAUC_lo":res["PRAUC"][1], "PRAUC_hi":res["PRAUC"][2],
        "Brier_med":res["Brier"][0], "Brier_lo":res["Brier"][1], "Brier_hi":res["Brier"][2],
    })

ci_df = pd.DataFrame(rows)
out = BASE/"p25_informe_final"/"p25_p24_test_bootstrap_ci.csv"
ci_df.to_csv(out, index=False)
print("Guardado:", out)
ci_df


Guardado: /content/drive/MyDrive/CognitivaAI/p25_informe_final/p25_p24_test_bootstrap_ci.csv


,Cohort,AUC_med,AUC_lo,AUC_hi,PRAUC_med,PRAUC_lo,PRAUC_hi,Brier_med,Brier_lo,Brier_hi
0,ALL,0.729184,0.606209,0.840000,0.727962,0.558498,0.858224,0.219830,0.195290,0.245109
1,OAS1,0.759058,0.597070,0.894444,0.755847,0.527241,0.888745,0.210051,0.181776,0.239942
2,OAS2,0.757576,0.516667,0.921569,0.820914,0.597870,0.950514,0.238513,0.195675,0.284057


In [32]:
# Expected Calibration Error (ECE) y Maximum Calibration Error (MCE), 10 bins cuantiles
import numpy as np, pandas as pd
from pathlib import Path

BASE = Path("/content/drive/MyDrive/CognitivaAI")
TEST = pd.read_csv(BASE/"p24_meta_simple"/"p24_test_preds.csv")

def ece_mce(y, p, n_bins=10):
    df = pd.DataFrame({"y":y, "p":p}).sort_values("p")
    df["bin"] = pd.qcut(df["p"].rank(method="first"), q=n_bins, labels=False)
    stats = df.groupby("bin").agg(n=("y","size"), mean_p=("p","mean"), acc=("y","mean")).reset_index()
    w = stats["n"]/stats["n"].sum()
    e = (w * (stats["acc"] - stats["mean_p"]).abs()).sum()
    m = (stats["acc"] - stats["mean_p"]).abs().max()
    return float(e), float(m)

rows=[]
for coh in ["ALL","OAS1","OAS2"]:
    part = TEST if coh=="ALL" else TEST[TEST["cohort"]==coh]
    y = part["y_true"].astype(int).values; p = part["y_prob"].astype(float).values
    e, m = ece_mce(y,p,n_bins=10)
    rows.append({"Cohort":coh, "ECE@10":e, "MCE@10":m})
ece_df = pd.DataFrame(rows)
out = BASE/"p25_informe_final"/"p25_p24_test_calibration_ece.csv"
ece_df.to_csv(out, index=False)
print("Guardado:", out)
ece_df


Guardado: /content/drive/MyDrive/CognitivaAI/p25_informe_final/p25_p24_test_calibration_ece.csv


,Cohort,ECE@10,MCE@10
0,ALL,0.122442,0.242288
1,OAS1,0.130507,0.236186
2,OAS2,0.294098,0.608658


In [33]:
from pathlib import Path
from textwrap import dedent

BASE = Path("/content/drive/MyDrive/CognitivaAI")
APP = dedent("""
## Apéndice — Decisión, calibración y estabilidad (P24)

**Coste vs. umbral (VAL):**
- Figuras: `p25_cost_curve_OAS1.png`, `p25_cost_curve_OAS2.png`
- Sensibilidad (3:1, 5:1, 7:1, 10:1): `p25_cost_sensitivity_p24.csv`

**Matriz de confusión (TEST @ thr por cohorte):**
- CSV: `p25_confusion_test_p24.csv`

**Curvas y calibración (TEST):**
- ROC/PR: `p25_roc_P24_TEST_OAS1.png`, `p25_roc_P24_TEST_OAS2.png`, `p25_pr_P24_TEST_OAS1.png`, `p25_pr_P24_TEST_OAS2.png`
- Calibración: `p25_cal_P24_TEST_OAS1.png`, `p25_cal_P24_TEST_OAS2.png`
- ECE/MCE: `p25_p24_test_calibration_ece.csv`

**Incertidumbre (bootstrap 95% CI):**
- `p25_p24_test_bootstrap_ci.csv`

**Interpretabilidad (LR elastic-net):**
- Top coeficientes: `p25_p24_top_coefficients.md` / `.csv`
""").strip()

out = BASE/"p25_informe_final"/"p25_appendix_links.md"
out.write_text(APP, encoding="utf-8")
print("Guardado:", out)
print(APP)


Guardado: /content/drive/MyDrive/CognitivaAI/p25_informe_final/p25_appendix_links.md
## Apéndice — Decisión, calibración y estabilidad (P24)

**Coste vs. umbral (VAL):**  
- Figuras: `p25_cost_curve_OAS1.png`, `p25_cost_curve_OAS2.png`  
- Sensibilidad (3:1, 5:1, 7:1, 10:1): `p25_cost_sensitivity_p24.csv`

**Matriz de confusión (TEST @ thr por cohorte):**  
- CSV: `p25_confusion_test_p24.csv`

**Curvas y calibración (TEST):**  
- ROC/PR: `p25_roc_P24_TEST_OAS1.png`, `p25_roc_P24_TEST_OAS2.png`, `p25_pr_P24_TEST_OAS1.png`, `p25_pr_P24_TEST_OAS2.png`  
- Calibración: `p25_cal_P24_TEST_OAS1.png`, `p25_cal_P24_TEST_OAS2.png`  
- ECE/MCE: `p25_p24_test_calibration_ece.csv`

**Incertidumbre (bootstrap 95% CI):**  
- `p25_p24_test_bootstrap_ci.csv`

**Interpretabilidad (LR elastic-net):**  
- Top coeficientes: `p25_p24_top_coefficients.md` / `.csv`
